In [0]:
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.transforms as transforms
import time

##Init Weights and Biases

In [0]:
%pip install wandb -q
import wandb
wandb.login()

True

## Define the transformations

In [0]:
# Define transforms for data preprocessing
transform = transforms.Compose([
    transforms.ToTensor()
])

trainset = torchvision.datasets.FashionMNIST(root='./data', train=True, download=True, transform=transform)

testset = torchvision.datasets.FashionMNIST(root='./data', train=False, download=True, transform=transform)

## Create data loaders to shuffle and create batches

In [0]:
trainloader = torch.utils.data.DataLoader(trainset, batch_size=1000, shuffle=True)

testloader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False)

In [0]:
print('No. of train images: {}'.format(len(trainset)))
print('No. of test images: {}'.format(len(testset)))

print('No. of train batches: {}'.format(len(trainloader)))
print('No. of test batches: {}'.format(len(testloader)))

No. of train images: 60000
No. of test images: 10000
No. of train batches: 60
No. of test batches: 100


## Create the Network

In [0]:
class Flatten(nn.Module):
    def forward(self, input):
        return input.view(input.size(0), -1)

flatten = Flatten()

def create_network_from_list(conv_list, linear_list):
    layers = []
    input_dim = 28

    for i in range(len(conv_list)-1):
        layers.append(nn.Conv2d(in_channels= conv_list[i], out_channels=conv_list[i+1], kernel_size=5))
        layers.append(nn.ReLU())
        input_dim = input_dim-4

    layers.append(flatten)
    layers.append(nn.Linear(conv_list[-1]*input_dim*input_dim, linear_list[0]))
    layers.append(nn.ReLU())

    for i in range(len(linear_list)-1):
        layers.append(nn.Linear(linear_list[i], linear_list[i+1]))
        if i == len(linear_list)-2:
            pass
        else:
            layers.append(nn.ReLU())

    return nn.Sequential(*layers)

# class Network(nn.Module):
    
#     def __init__(self):
#         super(Network, self).__init__()
    
#         self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5)
#         self.conv2 = nn.Conv2d(in_channels=6, out_channels=12, kernel_size=5)

#         self.pool = nn.MaxPool2d(kernel_size = 2, stride = 2)

#         self.fc1 = nn.Linear(12 * 4 * 4, 120)
#         self.fc2 = nn.Linear(120, 60)
#         self.fc3 = nn.Linear(60, 10)

#     def forward(self, x):
#         x = self.conv1(x)
#         x = F.relu(x)
#         x = self.pool(x)
        
#         x = self.conv2(x)
#         x = F.relu(x)
#         x = self.pool(x)
        
#         x = x.reshape(-1, 12 * 4 * 4)
        
#         x = self.fc1(x)
#         x = F.relu(x)
        
#         x = self.fc2(x)
#         x = F.relu(x)
        
#         x = self.fc3(x)
#         return x

## Function to find the accuracy

In [0]:
def find_acc(pred, label):
    correct = pred.argmax(dim = 1).eq(label)
    accuracy = correct.to(torch.float32).mean().item() * 100
    return accuracy

In [0]:
def train(network, epoch, criterion, optimizer, trainloader):
    loss_train = 0
    acc_train = 0
    network.train()
    
    for step in range(len(trainloader)):

        images , labels = next(iter(trainloader))
        
        # move the images and labels to GPU
        images = images.cuda()
        labels = labels.cuda()
        
        pred = network(images)
        
        # clear all the gradients before calculating them
        optimizer.zero_grad()
        
        # find the loss for the current step
        loss_train_step = criterion(pred , labels)
        
        # find accuracy
        acc_train_step = find_acc(pred, labels)
        
        # calculate the gradients
        loss_train_step.backward()
        
        # update the parameters
        optimizer.step()
        
        loss_train += loss_train_step.item()
        acc_train += acc_train_step  
            
    loss_train /= len(trainloader)
    acc_train /= len(testloader)

    return loss_train, acc_train  
        
def validate(network, epoch, criterion, optimizer, trainloader): 
    loss_valid = 0
    acc_valid = 0       
    network.eval()  

    for step in range(len(testloader)):

        images , labels = next(iter(testloader))
        
        # move the images and labels to GPU
        images = images.cuda()
        labels = labels.cuda()
        
        pred = network(images)
        
        # clear all the gradients before calculating them
        optimizer.zero_grad()
        
        # find the loss and acc for the current step
        loss_valid_step = criterion(pred , labels)
        acc_valid_step = find_acc(pred, labels)
      
        loss_valid += loss_valid_step.item()
        acc_valid += acc_valid_step

    loss_valid /= len(trainloader)
    acc_valid /= len(testloader)

    return loss_valid, acc_valid

## Train the Network

In [0]:
def train_hyperparameter_sweep():

    initial_config = {
        'learning_rate': 1,
        'epochs': 5,
        'conv_list': [1,6,12],
        'linear_list': [120,60,10]
    }

    # WandB – Initialize a new run
    wandb.init(name='hyperparameter_sweep', notes='This is a trial',entity="arkalim", project="pytorch-fashion_mnist", config=initial_config)
    # wandb.watch_called = False # Re-run the model without restarting the runtime, unnecessary after our next release 

    # create an instance of the Network    
    network = create_network_from_list(wandb.config.conv_list, wandb.config.linear_list).cuda()
    # network = Network().cuda()

    # loss defined using torch.nn
    criterion = nn.CrossEntropyLoss()

    # define the optimizer
    optimizer = optim.Adam(network.parameters(), wandb.config.learning_rate)

    # note the starting time to find the total time elapsed
    start_time = time.time()

    for epoch in range(1,wandb.config.epochs+1):
        
        loss_train, acc_train = train(network, epoch, criterion, optimizer, trainloader)
        loss_valid, acc_valid = validate(network, epoch, criterion, optimizer, testloader)
        
        print('Epoch: {}  Train Loss: {}  Train Acc: {}  Valid Loss: {}  Valid Acc: {}'.format(epoch, loss_train, acc_train, loss_valid, acc_valid))

        wandb.log({
            "Train Loss": loss_train,
            "Train Acc": acc_train,
            "Valid Loss": loss_valid,
            "Valid Acc": acc_valid})
        
    # find the time at the end of training    
    end_time = time.time()

    total_time = end_time - start_time
    print("Total time taken : {}".format(total_time))

In [0]:
sweep_config = {
    'method': 'grid',
    'metric': {
      'name': 'Valid Acc',
      'goal': 'maximize'   
    },
    'parameters': {
        'learning_rate': {
            'values': [0.01, 0.001]
        },
        'epochs': {
            'values': [25]
        },
        'conv_list': {
            'values': [[1,4,8], [1,6,12], [1,12,24]]
        },
        'linear_list': {
            'values': [[20,40,10], [120,60,10], [240,120,10]]
        }
    }
}

sweep_id = wandb.sweep(sweep_config)

Create sweep with ID: m861u4gd
Sweep URL: https://app.wandb.ai/arkalim/uncategorized/sweeps/m861u4gd


In [0]:
wandb.agent(sweep_id, function=train_hyperparameter_sweep)

wandb: Agent Starting Run: 3ouwpfzo with config:
	conv_list: [1, 4, 8]
	epochs: 25
	learning_rate: 0.01
	linear_list: [20, 40, 10]
wandb: Agent Started Run: 3ouwpfzo


wandb: WARNING Ignoring project='pytorch-fashion_mnist' passed to wandb.init when running a sweep


Epoch: 1  Train Loss: 0.9074445297320684  Train Acc: 39.47300183773041  Valid Loss: 0.6987519264221191  Valid Acc: 75.99999904632568
Epoch: 2  Train Loss: 0.48528456389904023  Train Acc: 48.93700224161148  Valid Loss: 0.550904393196106  Valid Acc: 78.99999618530273
Epoch: 3  Train Loss: 0.4137252449989319  Train Acc: 50.91500240564346  Valid Loss: 0.48391568660736084  Valid Acc: 81.00000023841858
Epoch: 4  Train Loss: 0.3670372739434242  Train Acc: 51.994002521038055  Valid Loss: 0.4126817584037781  Valid Acc: 87.99999952316284
Epoch: 5  Train Loss: 0.34050183643897375  Train Acc: 52.496002435684204  Valid Loss: 0.4271571636199951  Valid Acc: 87.99999952316284
Epoch: 6  Train Loss: 0.3191362500190735  Train Acc: 53.01600247621536  Valid Loss: 0.4253605008125305  Valid Acc: 87.00000047683716
Epoch: 7  Train Loss: 0.3096292977531751  Train Acc: 53.14200246334076  Valid Loss: 0.4409109354019165  Valid Acc: 84.99999642372131
Epoch: 8  Train Loss: 0.29251224795977276  Train Acc: 53.53300231

wandb: WARNING Ignoring project='pytorch-fashion_mnist' passed to wandb.init when running a sweep


Epoch: 1  Train Loss: 0.813479853173097  Train Acc: 40.97300187498331  Valid Loss: 0.5425302386283875  Valid Acc: 76.99999809265137
Epoch: 2  Train Loss: 0.43185092359781263  Train Acc: 50.282002329826355  Valid Loss: 0.42145609855651855  Valid Acc: 82.99999833106995
Epoch: 3  Train Loss: 0.35625524719556173  Train Acc: 52.09100252389908  Valid Loss: 0.4255433976650238  Valid Acc: 84.99999642372131
Epoch: 4  Train Loss: 0.3185265416900317  Train Acc: 52.87700283527374  Valid Loss: 0.3787592351436615  Valid Acc: 87.00000047683716
Epoch: 5  Train Loss: 0.2752816520631313  Train Acc: 53.87100261449814  Valid Loss: 0.44208961725234985  Valid Acc: 85.999995470047
Epoch: 6  Train Loss: 0.24878066976865132  Train Acc: 54.43400251865387  Valid Loss: 0.43551188707351685  Valid Acc: 83.99999737739563
Epoch: 7  Train Loss: 0.23892408857742944  Train Acc: 54.652002930641174  Valid Loss: 0.39769843220710754  Valid Acc: 85.999995470047
Epoch: 8  Train Loss: 0.210806888093551  Train Acc: 55.333002686

...

Epoch: 1  Train Loss: 1.0181377559900284  Train Acc: 37.93000192195177  Valid Loss: 0.6376911401748657  Valid Acc: 75.99999904632568
Epoch: 2  Train Loss: 0.5411930600802104  Train Acc: 47.6490021944046  Valid Loss: 0.5781899094581604  Valid Acc: 76.99999809265137
Epoch: 3  Train Loss: 0.4766893724600474  Train Acc: 49.65500235557556  Valid Loss: 0.5321648716926575  Valid Acc: 81.99999928474426
Epoch: 4  Train Loss: 0.42549639840920767  Train Acc: 50.833002388477325  Valid Loss: 0.5103874802589417  Valid Acc: 83.99999737739563
Epoch: 5  Train Loss: 0.39755013634761177  Train Acc: 51.47600245475769  Valid Loss: 0.4629420042037964  Valid Acc: 84.99999642372131
Epoch: 6  Train Loss: 0.3760975663860639  Train Acc: 51.85800260305405  Valid Loss: 0.4550953805446625  Valid Acc: 85.999995470047
Epoch: 7  Train Loss: 0.35284629414478935  Train Acc: 52.31100249290466  Valid Loss: 0.444590300321579  Valid Acc: 87.99999952316284
Epoch: 8  Train Loss: 0.3393288026253382  Train Acc: 52.6400025486946

wandb: WARNING Ignoring project='pytorch-fashion_mnist' passed to wandb.init when running a sweep


Epoch: 1  Train Loss: 0.89692256351312  Train Acc: 41.16800181567669  Valid Loss: 0.6583616137504578  Valid Acc: 75.0
Epoch: 2  Train Loss: 0.49627720067898434  Train Acc: 49.06600230932236  Valid Loss: 0.5284517407417297  Valid Acc: 81.00000023841858
Epoch: 3  Train Loss: 0.41902002741893135  Train Acc: 50.92200243473053  Valid Loss: 0.5000936388969421  Valid Acc: 83.99999737739563
Epoch: 4  Train Loss: 0.3791312317053477  Train Acc: 51.765002489089966  Valid Loss: 0.47709617018699646  Valid Acc: 85.999995470047
Epoch: 5  Train Loss: 0.3528997008999189  Train Acc: 52.351002395153046  Valid Loss: 0.4789575934410095  Valid Acc: 85.999995470047
Epoch: 6  Train Loss: 0.32927943815787636  Train Acc: 52.69500243663788  Valid Loss: 0.4860702455043793  Valid Acc: 84.99999642372131
Epoch: 7  Train Loss: 0.3085961972673734  Train Acc: 53.279002487659454  Valid Loss: 0.4225066006183624  Valid Acc: 84.99999642372131
Epoch: 8  Train Loss: 0.28774108017484346  Train Acc: 53.68000257015228  Valid Lo

wandb: WARNING Ignoring project='pytorch-fashion_mnist' passed to wandb.init when running a sweep


Epoch: 1  Train Loss: 0.8504361003637314  Train Acc: 41.4960020929575  Valid Loss: 0.6490694284439087  Valid Acc: 75.0
Epoch: 2  Train Loss: 0.4747125953435898  Train Acc: 49.532002389431  Valid Loss: 0.5205243825912476  Valid Acc: 79.99999523162842
Epoch: 3  Train Loss: 0.4071669856707255  Train Acc: 51.09300243854523  Valid Loss: 0.5324647426605225  Valid Acc: 83.99999737739563
Epoch: 4  Train Loss: 0.35596311539411546  Train Acc: 52.22600257396698  Valid Loss: 0.42566078901290894  Valid Acc: 84.99999642372131
Epoch: 5  Train Loss: 0.3216888099908829  Train Acc: 52.79300236701965  Valid Loss: 0.43052390217781067  Valid Acc: 85.999995470047
Epoch: 6  Train Loss: 0.29941621323426565  Train Acc: 53.38800257444382  Valid Loss: 0.36788275837898254  Valid Acc: 85.999995470047
Epoch: 7  Train Loss: 0.27517860432465874  Train Acc: 54.0110028386116  Valid Loss: 0.38392502069473267  Valid Acc: 84.99999642372131
Epoch: 8  Train Loss: 0.2627205319702625  Train Acc: 54.11700260639191  Valid Loss: